<a href="https://colab.research.google.com/github/jjmatthiesen/homeOffice/blob/main/HomeOffice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygwalker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.5/707.5 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.8 MB/s eta 0:00:00


In [ ]:
import pygwalker as pyg
import pandas as pd

from IPython.display import Image
import numpy as np
import datetime
import warnings
import pathlib

In [ ]:
def load_data(comp):
    d_employees = pd.read_csv('./data/' + comp + '/employees_' + comp + '.csv', sep=',')
    d_tasks = pd.read_csv('./data/' + comp + '/tasks_' + comp + '.csv', sep=',')
    d_employee_input = pd.read_csv('./data/' + comp + '/employee_input_' + comp + '.csv', sep=',')
    return d_employees, d_tasks, d_employee_input

# calculate diff between today and the start date at the company
def tenure(start_date):
    date_now = datetime.datetime.utcnow()
    date_diff = int((date_now - start_date).days)
    return date_diff


def ho_prob(days):
    if days < 180:
        home_office_prob = 0
    else:
        home_office_prob = 100
    return home_office_prob


# calculate the home office level based on the age (gen-preference)
def ho_generation(year):
    if 1946 <= year <= 1964:
        return 48
    elif 1965 <= year <= 1980:
        return 50
    elif 1981 <= year <= 1994:
        return 44
    else:
        return 28


# calculate the home office level based on the education
def ho_degree(abschluss):
    if abschluss == 'Hochschule':
        return 48
    elif abschluss == 'mittlere Reife':
        return 17
    else:
        return 8


# calculate the home office level based on the commute time
def ho_commute(time):
    if time > 40:
        return 46
    elif 20 <= time <= 40:
        return 2.3 * time -46 # linear decline
    else:
        return 0


# calculate the home office level based on caring responsibility
def ho_gender_resp(employees):
    if employees['Fürsorgeverantwortung'] and employees['Geschlecht'] == 'weiblich':
        return 50
    elif employees['Fürsorgeverantwortung'] and employees['Geschlecht'] == 'männlich':
        return 48
    else:
        return 46


# calculate the preferred level of home office amount
def ho_prefer(employee_input):
    if employee_input['ho_wunsch'] == True:
        return (employee_input['wunsch_tage'] / 5) * 100
    else:
        return 0


# calculate derivations
def deviation(ho_shares):
    ho_shares['difference_to_max'] = int(ho_shares['ho_max'] - ho_shares['ho_prefer'])
    ho_shares['difference_to_opt'] = int(ho_shares['ho_opt'] - ho_shares['ho_prefer'])
    ho_shares['difference_to_social'] = int(ho_shares['ho_social'] - ho_shares['ho_prefer'])
    return ho_shares


In [ ]:
# this will work once we have the data public
#employees, tasks, employee_input = load_data('example)

['Personalnummer', 'Arbeitnehmer', 'ho_wunsch', 'wunsch_tage', 'Fürsorgeverantwortung', 'Pendelzeit']


In [ ]:
employees = pd.DataFrame([['111111', 'AA', '1962-02-23', '2000-07-01', 'männlich', 'Hochschule', 'Personalleiter'], ['111112', 'BB', '1996-09-16', '2021-02-01', 'weiblich', 'mittlere Reife', 'Einkaufsleiter'], ['111113', 'CC', '1988-04-12', '2018-10-01', 'weiblich', 'Hochschule', 'Sekretär'], ['111114', 'DD', '1973-12-31', '1995-01-01', 'weiblich', 'Hauptschulabschluß', 'Hausmeister'], ['111115', 'EE', '1986-07-03', '2010-12-01', 'männlich', 'Hauptschulabschluß', 'Reinigungskraft'], ['111116', 'FF', '1965-04-19', '2003-04-01', 'männlich', 'Hochschule', 'IT Fachkraft 1'], ['111117', 'GG', '1977-09-23', '2008-05-01', 'männlich', 'Hochschule', 'Controller'], ['111118', 'HH', '1971-10-26', '2014-06-01', 'weiblich', 'Hochschule', 'IT Fachkraft 2'], ['111119', 'II', '1994-11-02', '2020-08-01', 'weiblich', 'Hochschule', 'Marketing Manager'], ['111120', 'JJ', '1958-05-16', '1996-11-01', 'weiblich', 'Hochschule', 'Teamleiter - Finanzen'], ['111121', 'KK', '1981-04-23', '2021-06-01', 'männlich', 'mittlere Reife', 'Vertriebsmitarbeiter'], ['111122', 'LL', '1970-09-21', '2010-06-01', 'weiblich', 'mittlere Reife', 'Filialleiter'], ['111123', 'MM', '1988-12-22', '2020-03-01', 'männlich', 'Hauptschulabschluß', 'Filialleiter'], ['111124', 'NN', '1993-08-02', '1999-11-01', 'weiblich', 'Hochschule', 'Verkäufer'], ['111125', 'OO', '1959-06-16', '2021-06-01', 'männlich', 'Hochschule', 'Verkäufer'], ['111126', 'PP', '1983-01-23', '2020-04-01', 'männlich', 'mittlere Reife', 'Verkäufer'], ['111127', 'QQ', '1990-02-21', '1996-12-01', 'weiblich', 'Hauptschulabschluß', 'Verkäufer']], columns=['Personalnummer', 'Arbeitnehmer', 'Geburtsdatum', 'Eintrittsdatum', 'Geschlecht', 'Bildungsabschluss', 'Tätigkeit'])
employees

Personalnummer Arbeitnehmer Geburtsdatum Eintrittsdatum Geschlecht  \
0          111111           AA   1962-02-23     2000-07-01   männlich   
1          111112           BB   1996-09-16     2021-02-01   weiblich   
2          111113           CC   1988-04-12     2018-10-01   weiblich   
3          111114           DD   1973-12-31     1995-01-01   weiblich   
4          111115           EE   1986-07-03     2010-12-01   männlich   
5          111116           FF   1965-04-19     2003-04-01   männlich   
6          111117           GG   1977-09-23     2008-05-01   männlich   
7          111118           HH   1971-10-26     2014-06-01   weiblich   
8          111119           II   1994-11-02     2020-08-01   weiblich   
9          111120           JJ   1958-05-16     1996-11-01   weiblich   
10         111121           KK   1981-04-23     2021-06-01   männlich   
11         111122           LL   1970-09-21     2010-06-01   weiblich   
12         111123           MM   1988-12-22     2020-03-01   männlich   
13         111124           NN   1993-08-02     1999-11-01   weiblich   
14         111125           OO   1959-06-16     2021-06-01   männlich   
15         111126           PP   1983-01-23     2020-04-01   männlich   
16         111127           QQ   1990-02-21     1996-12-01   weiblich   

     Bildungsabschluss              Tätigkeit  
0           Hochschule         Personalleiter  
1       mittlere Reife         Einkaufsleiter  
2           Hochschule               Sekretär  
3   Hauptschulabschluß            Hausmeister  
4   Hauptschulabschluß        Reinigungskraft  
5           Hochschule         IT Fachkraft 1  
6           Hochschule             Controller  
7           Hochschule         IT Fachkraft 2  
8           Hochschule      Marketing Manager  
9           Hochschule  Teamleiter - Finanzen  
10      mittlere Reife   Vertriebsmitarbeiter  
11      mittlere Reife           Filialleiter  
12  Hauptschulabschluß           Filialleiter  
13          Hochschule              Verkäufer  
14          Hochschule              Verkäufer  
15      mittlere Reife              Verkäufer  
16  Hauptschulabschluß              Verkäufer

In [ ]:
tasks = pd.DataFrame([['Personalleiter', '0', '0', '0', '0', '0', '0', '0', '0', '90', '0', '0', '0', '0', '0', '10', '0', '10', '40', '30', '10', '0', 'True'], ['Einkaufsleiter', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '10', '10', '80', '30', '20', '30', '0', '0', 'True'], ['Sekretär', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '50', '10', '40', '0', '40', '10', '0', '0', '0', 'True'], ['Hausmeister', '0', '80', '0', '0', '0', '0', '0', '10', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'False'], ['Reinigungskraft', '0', '0', '0', '0', '0', '0', '0', '100', '0', '0', '0', '0', '0', '10', '0', '0', '0', '0', '0', '0', '0', 'False'], ['IT Fachkraft 1', '0', '0', '0', '0', '0', '0', '0', '0', '20', '0', '0', '0', '0', '50', '30', '0', '10', '10', '0', '0', '0', 'True'], ['Controller', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '10', '0', '40', '30', '20', '0', '30', '20', '0', '0', '0', 'False'], ['IT Fachkraft 2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '65', '30', '5', '5', '0', '0', '0', '0', 'True'], ['Marketing Manager', '0', '0', '0', '0', '0', '0', '0', '0', '0', '80', '15', '0', '0', '5', '0', '0', '5', '0', '0', '0', '0', 'True'], ['Teamleiter - Finanzen', '0', '0', '0', '0', '0', '0', '0', '0', '30', '0', '40', '0', '10', '10', '10', '0', '30', '20', '30', '0', '0', 'True'], ['Vertriebsmitarbeiter', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '10', '10', '10', '70', '5', '10', '50', '5', '10', 'True'], ['Filialleiter', '0', '15', '0', '0', '0', '0', '0', '0', '20', '0', '40', '0', '5', '5', '10', '0', '50', '10', '50', '10', '0', 'True'], ['Verkäufer', '0', '20', '0', '0', '0', '0', '0', '0', '70', '0', '0', '0', '0', '5', '5', '0', '0', '0', '50', '0', '40', 'False']], columns=['Tätigkeit', 'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'Q01', 'Q02', 'Q03', 'Q41', 'Q42', 'it'])
tasks.loc[:, "T01":"Q42"] = tasks.loc[:,"T01":"Q42"].apply(pd.to_numeric)
tasks

Tätigkeit T01 T02 T03 T04 T05 T06 T07  T08 T09  ... T13 T14  \
0          Personalleiter   0   0   0   0   0   0   0    0  90  ...   0   0   
1          Einkaufsleiter   0   0   0   0   0   0   0    0   0  ...   0  10   
2                Sekretär   0   0   0   0   0   0   0    0   0  ...  50  10   
3             Hausmeister   0  80   0   0   0   0   0   10   0  ...   0   0   
4         Reinigungskraft   0   0   0   0   0   0   0  100   0  ...   0  10   
5          IT Fachkraft 1   0   0   0   0   0   0   0    0  20  ...   0  50   
6              Controller   0   0   0   0   0   0   0    0   0  ...  40  30   
7          IT Fachkraft 2   0   0   0   0   0   0   0    0   0  ...   0  65   
8       Marketing Manager   0   0   0   0   0   0   0    0   0  ...   0   5   
9   Teamleiter - Finanzen   0   0   0   0   0   0   0    0  30  ...  10  10   
10   Vertriebsmitarbeiter   0   0   0   0   0   0   0    0   0  ...  10  10   
11           Filialleiter   0  15   0   0   0   0   0    0  20  ...   5   5   
12              Verkäufer   0  20   0   0   0   0   0    0  70  ...   0   5   

   T15 T16 Q01 Q02 Q03 Q41 Q42     it  
0   10   0  10  40  30  10   0   True  
1   10  80  30  20  30   0   0   True  
2   40   0  40  10   0   0   0   True  
3    0   0   0   0   0   0   0  False  
4    0   0   0   0   0   0   0  False  
5   30   0  10  10   0   0   0   True  
6   20   0  30  20   0   0   0  False  
7   30   5   5   0   0   0   0   True  
8    0   0   5   0   0   0   0   True  
9   10   0  30  20  30   0   0   True  
10  10  70   5  10  50   5  10   True  
11  10   0  50  10  50  10   0   True  
12   5   0   0   0  50   0  40  False  

[13 rows x 23 columns]

In [ ]:
employee_input = pd.DataFrame([['111111', 'AA', 'True', '2', 'True', '40'], ['111112', 'BB', 'False', '3', 'True', '120'], ['111113', 'CC', 'True', '3', 'True', '10'], ['111114', 'DD', 'False', '0', 'False', '20'], ['111115', 'EE', 'False', '0', 'False', '50'], ['111116', 'FF', 'True', '4', 'True', '30'], ['111117', 'GG', 'True', '2', 'True', '25'], ['111118', 'HH', 'True', '5', 'False', '60'], ['111119', 'II', 'True', '2', 'True', '45'], ['111120', 'JJ', 'False', '0', 'False', '70'], ['111121', 'KK', 'False', '4', 'True', '30'], ['111122', 'LL', 'True', '1', 'True', '10'], ['111123', 'MM', 'True', '2', 'False', '40'], ['111124', 'NN', 'False', '2', 'False', '20'], ['111125', 'OO', 'False', '1', 'False', '80'], ['111126', 'PP', 'True', '0', 'True', '35'], ['111127', 'QQ', 'False', '2', 'True', '90']], columns=['Personalnummer', 'Arbeitnehmer', 'ho_wunsch', 'wunsch_tage', 'Fürsorgeverantwortung', 'Pendelzeit'])

employee_input[["wunsch_tage", "Pendelzeit"]] = employee_input[["wunsch_tage", "Pendelzeit"]].apply(pd.to_numeric)
employee_input


Personalnummer Arbeitnehmer ho_wunsch  wunsch_tage Fürsorgeverantwortung  \
0          111111           AA      True            2                  True   
1          111112           BB     False            3                  True   
2          111113           CC      True            3                  True   
3          111114           DD     False            0                 False   
4          111115           EE     False            0                 False   
5          111116           FF      True            4                  True   
6          111117           GG      True            2                  True   
7          111118           HH      True            5                 False   
8          111119           II      True            2                  True   
9          111120           JJ     False            0                 False   
10         111121           KK     False            4                  True   
11         111122           LL      True            1                  True   
12         111123           MM      True            2                 False   
13         111124           NN     False            2                 False   
14         111125           OO     False            1                 False   
15         111126           PP      True            0                  True   
16         111127           QQ     False            2                  True   

    Pendelzeit  
0           40  
1          120  
2           10  
3           20  
4           50  
5           30  
6           25  
7           60  
8           45  
9           70  
10          30  
11          10  
12          40  
13          20  
14          80  
15          35  
16          90

In [ ]:
# -------------------------
# Employer's point of view
# -------------------------

# take sum of tasks
# 3.2.1 The Teleworkability-Index
ho_tasks = tasks['T09'] + tasks['T10'] + tasks['T11'] + tasks['T12'] + tasks['T13'] + tasks['T14'] + tasks['T15'] +  tasks['T16']

# add new column to df
tasks['ho_max'] = ho_tasks
tasks

Tätigkeit T01 T02 T03 T04 T05 T06 T07  T08 T09  ... T14 T15  \
0          Personalleiter   0   0   0   0   0   0   0    0  90  ...   0  10   
1          Einkaufsleiter   0   0   0   0   0   0   0    0   0  ...  10  10   
2                Sekretär   0   0   0   0   0   0   0    0   0  ...  10  40   
3             Hausmeister   0  80   0   0   0   0   0   10   0  ...   0   0   
4         Reinigungskraft   0   0   0   0   0   0   0  100   0  ...  10   0   
5          IT Fachkraft 1   0   0   0   0   0   0   0    0  20  ...  50  30   
6              Controller   0   0   0   0   0   0   0    0   0  ...  30  20   
7          IT Fachkraft 2   0   0   0   0   0   0   0    0   0  ...  65  30   
8       Marketing Manager   0   0   0   0   0   0   0    0   0  ...   5   0   
9   Teamleiter - Finanzen   0   0   0   0   0   0   0    0  30  ...  10  10   
10   Vertriebsmitarbeiter   0   0   0   0   0   0   0    0   0  ...  10  10   
11           Filialleiter   0  15   0   0   0   0   0    0  20  ...   5  10   
12              Verkäufer   0  20   0   0   0   0   0    0  70  ...   5   5   

   T16 Q01 Q02 Q03 Q41 Q42     it ho_max  
0    0  10  40  30  10   0   True    100  
1   80  30  20  30   0   0   True    100  
2    0  40  10   0   0   0   True    100  
3    0   0   0   0   0   0  False      0  
4    0   0   0   0   0   0  False     10  
5    0  10  10   0   0   0   True    100  
6    0  30  20   0   0   0  False    100  
7    5   5   0   0   0   0   True    100  
8    0   5   0   0   0   0   True    100  
9    0  30  20  30   0   0   True    100  
10  70   5  10  50   5  10   True    100  
11   0  50  10  50  10   0   True     80  
12   0   0   0  50   0  40  False     80  

[13 rows x 24 columns]

In [ ]:
# 3.2.2 Infrastructure.
tasks.loc[tasks.it == False, ['ho_max']] = 0

# 3.2.3 Sense of Belonging to Company.
employees['Eintrittsdatum'] = pd.to_datetime(employees['Eintrittsdatum'])
employees['Unternehmenszugehörigkeit'] = employees['Eintrittsdatum'].apply(tenure)
employees['ho_prob'] = employees['Unternehmenszugehörigkeit'].apply(ho_prob)

tasks = pd.merge(tasks, employees[['Tätigkeit', 'ho_prob']], on='Tätigkeit', how='left')
tasks.loc[tasks.ho_prob == 0, ['ho_max']] = 0
# calculate the mean
ho_max_total = int((int(tasks['ho_max'].sum())) / len(tasks.index))

# results after first 3 steps (mean)
print('The maximum level of home office at the company after the first three points is: ' + str(ho_max_total) + '%.')

The maximum level of home office at the company after the first three points is: 81%.


In [ ]:
# 3.2.4 Task-Media-Fit Model
grouptasks_ho = tasks['Q01'] + tasks['Q02'] + tasks['Q42']
grouptasks_office = tasks['Q03'] + tasks['Q41']

tasks['grouptasks_ho'] = grouptasks_ho
tasks['grouptasks_office'] = grouptasks_office
tasks[['Tätigkeit','grouptasks_ho', 'grouptasks_office']]

opt_tasks = tasks['ho_max'] - tasks['grouptasks_office']
tasks['ho_opt'] = opt_tasks

# correct if mox amount of home office was already zero:
tasks.loc[tasks.ho_opt <= 0, ['ho_opt']] = 0

# calculate the mean
ho_opt_total = int((int(tasks['ho_opt'].sum())) / len(tasks.index))
print('The optimal amount of home office based on the media fit model is: ' + str(ho_opt_total) + '%.')

The optimal amount of home office based on the media fit model is: 53%.


In [ ]:
# -------------------------
# Social factors
# -------------------------

# 3.3.1 Different generations
employees['Geburtsdatum'] = pd.to_datetime(employees['Geburtsdatum'])
employees['birth_year'] = employees['Geburtsdatum'].dt.year
employees['ho_generation'] = employees['birth_year'].apply(ho_generation)

# 3.3.2 Education.
employees['ho_degree'] = employees['Bildungsabschluss'].apply(ho_degree)

# 3.3.3 Commute time
employees['ho_commute'] = employee_input['Pendelzeit'].apply(ho_commute)

# 3.3.4 Caring Responsibility
employees = employees.join(employee_input['Fürsorgeverantwortung'])
employees['ho_responsibility'] = employees.apply(ho_gender_resp, axis=1)

# Social factors subset
employees_subset = employees[['ho_generation', 'ho_degree', 'ho_commute', 'ho_responsibility']]

# calculate the mean
average_value_ho = np.average(employees_subset, axis=1)
employees_subset['ho_social'] = average_value_ho
ho_social_total = round((int(employees_subset['ho_social'].sum()) / len(employees.index)), 2)
print('The optimal level of home office based on the social factors is: ' + str(ho_social_total) + '%.')

The optimal amount of home office based on the social factors is: 36.18%.


<ipython-input-50-9b2beb3f761b>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employees_subset['ho_social'] = average_value_ho


In [ ]:
# %%
# -------------------------
# Employee Requests
# -------------------------

# 3.4 Employee Requests
employee_input['ho_prefer'] = employee_input.apply(ho_prefer, axis=1)

# calculate the mean
ho_prefer_total = round((int(employee_input['ho_prefer'].sum()) / len(employee_input.index)), 2)
print('The mean of the employees wishes for home office is: ' + str(ho_prefer_total) + '%.')

The mean of the employees wishes for home office is: 0.0%.


In [ ]:
# %%
# -------------------------
# Difference Between the Calculated level of Home Office and Preference
# -------------------------

employees = employees.join(employee_input['ho_prefer'])
employees = employees.join(employees_subset['ho_social'])
employees = pd.merge(employees, tasks[['Tätigkeit', 'ho_max']], on='Tätigkeit', how='left')
employees = pd.merge(employees, tasks[['Tätigkeit', 'ho_opt']], on='Tätigkeit', how='left')

ho_shares = employees[['Personalnummer', 'Tätigkeit', 'Arbeitnehmer', 'ho_social', 'ho_opt', 'ho_max', 'ho_prefer']]
ho_shares = ho_shares.drop_duplicates()
employees = employees.drop_duplicates()

ho_shares = ho_shares.apply(deviation, axis=1)
ho_shares

Personalnummer              Tätigkeit Arbeitnehmer  ho_social  ho_opt  \
0          111111         Personalleiter           AA      36.00      60   
1          111112         Einkaufsleiter           BB      35.25      70   
2          111113               Sekretär           CC      35.50     100   
3          111114            Hausmeister           DD      27.00       0   
4          111115        Reinigungskraft           EE      36.50      10   
5          111116         IT Fachkraft 1           FF      36.50     100   
6          111117             Controller           GG      36.50     100   
7          111118         IT Fachkraft 2           HH      48.50     100   
8          111119      Marketing Manager           II      47.00     100   
9          111120  Teamleiter - Finanzen           JJ      48.00      70   
10         111121   Vertriebsmitarbeiter           KK      27.25      45   
11         111122           Filialleiter           LL      29.25      20   
15         111123           Filialleiter           MM      25.00      20   
19         111124              Verkäufer           NN      35.50      30   
35         111125              Verkäufer           OO      47.50      30   
51         111126              Verkäufer           PP      27.25      30   
67         111127              Verkäufer           QQ      37.00      30   

    ho_max  ho_prefer  difference_to_max  difference_to_opt  \
0      100          0                100                 60   
1      100          0                100                 70   
2      100          0                100                100   
3        0          0                  0                  0   
4       10          0                 10                 10   
5      100          0                100                100   
6      100          0                100                100   
7      100          0                100                100   
8      100          0                100                100   
9      100          0                100                 70   
10     100          0                100                 45   
11      80          0                 80                 20   
15      80          0                 80                 20   
19      80          0                 80                 30   
35      80          0                 80                 30   
51      80          0                 80                 30   
67      80          0                 80                 30   

    difference_to_social  
0                     36  
1                     35  
2                     35  
3                     27  
4                     36  
5                     36  
6                     36  
7                     48  
8                     47  
9                     48  
10                    27  
11                    29  
15                    25  
19                    35  
35                    47  
51                    27  
67                    37

In [ ]:
pyg.walk(ho_shares)

In [ ]:
!pip install dtale

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 KB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 86.5 MB/s eta 0:00:00
  Using cached con

In [ ]:
### just to test out

import dtale

d = dtale.show(ho_shares)
d.open_browser()

ImportError: ignored